## Extracting the text from the image.
Using a ocr model to extract the text from the image.

In [ ]:
!pip install qwen_vl_utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 18.2 MB/s eta 0:00:00


In [ ]:
!pip install litellm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.4 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade sympy

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import os
import utils
import time
from litellm import completion
import litellm
import json
import re
from datetime import datetime

In [ ]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "prithivMLmods/Qwen2-VL-OCR-2B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
processor = AutoProcessor.from_pretrained("prithivMLmods/Qwen2-VL-OCR-2B-Instruct")

_prompt="""
Extract the overall text written in the image. Also do not extract any incorrect text from the image.
"""

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.42G [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


generation_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/408 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
def extracted_text(url,_prompt):
    messages = [
      {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": url,
            },
            {
                "type": "text",
                "text": _prompt,
            },
          ],
      }
    ]
    text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=False,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    for text in output_text:
      cleaned_outputs = text.replace("<|im_end|>", "").replace("\n"," ").strip()

    return cleaned_outputs


In [ ]:
prid_list=[]
documents_with_ocr=utils.load_jsonl("/content/product_list2.jsonl")
for document in documents_with_ocr:
  url=document["url"]
  prid=url.split("/")[-1]
  prid_list.append(prid)
print(len(prid_list))



100


In [ ]:
documents=utils.load_jsonl("/content/updated_products.jsonl")

In [ ]:
batch_size = 1
batch = []

for document in documents:
    t1 = time.time()
    url = document["url"]
    prid = url.split("/")[-1]

    if prid not in prid_list:
        image_bucket_urls = document["image_bucket_urls"]
        responses_lst = []

        for image in image_bucket_urls:
            text = extracted_text(image["url"], _prompt)
            image["raw_ocr_text"] = text

        batch.append(document)

        if len(batch) == batch_size:
            for doc in batch:
                utils.save_jsonl(output_file, doc)
            print(f"Processed batch of {batch_size} documents.")
            batch = []
        print("Time taken for this document:", time.time() - t1)

if batch:
    for doc in batch:
        utils.save_jsonl("/content/product_list2.jsonl", doc)
    print(f"Processed the final batch of {len(batch)} documents.")


OutOfMemoryError: CUDA out of memory. Tried to allocate 21.62 GiB. GPU 0 has a total capacity of 14.74 GiB of which 6.08 GiB is free. Process 2777 has 8.66 GiB memory in use. Of the allocated memory 7.65 GiB is allocated by PyTorch, and 909.71 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)